In [ ]:
from pyspark import SparkConf
from pyspark.context import SparkContext

In [ ]:
sc = SparkContext.getOrCreate(SparkConf())
lines = sc.textFile("../../data/pl.csv")
line_token_count = lines.map(lambda line: len(line.split(",")))
total_token_count = line_token_count.reduce(lambda a,b: a + b)
print(total_token_count)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
from pandas import json_normalize
import pandas as pd
import json

data = json.load(open('../../data/pl.json'))
dataframe = json_normalize(data)
dataframe = dataframe.astype(dtype= {
      "languageLabel" : "<U200", "year" : "int64"})

df = spark.createDataFrame(dataframe)

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df.show(1)

In [ ]:
def pandas_filter_func(iterator):
    for pandas_df in iterator:
        yield pandas_df[pandas_df.year == 1952]

df.mapInPandas(pandas_filter_func, schema=df.schema).show()

In [ ]:
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series + 1

df.select(pandas_plus_one(df.year)).show()

In [ ]:
df.groupby('year').count().show()

In [ ]:
df.show()